# Task Breakdown

- Data Collection: Data is collected and provided to you.
- Data Pre-processing: Convert the musical scores into a format suitable for deep learning models. This involves converting the musical scores into MIDI files and applying data augmentation techniques.
- Feature Extraction: Extractfeatures from the MIDI files, such as notes, chords, and tempo, using music analysis tools.
- Model Building: Develop a deep learning model using LSTM and CNN architectures to classify the musical scores according to the composer.
- Model Training: Train the deep learning model using the pre-processed and feature-extracted data.
- Model Evaluation: Evaluate the performance of the deep learning model using accuracy, precision, and recall metrics.
- Model Optimization: Optimize the deep learning model by fine-tuning hyperparameters.

In [16]:
!pip install pretty_midi

Defaulting to user installation because normal site-packages is not writeable


In [17]:
# Functions for Data Collection
# Function for Feature extractions
import os
import glob
import pretty_midi
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Check if pickle file exists and use the file for dataset
import pickle


# Directory of MIDI files
base_dir = 'train'

# Specify the path and filename of the pickle file
# Since the pkl file exists, that means the data was processed and features extracted
pickle_file_name = 'team8_composer_dataset.pkl'

# TBD:Extract features using librosa for further feature extraction
def calculate_features(midi_file):
    # Load MIDI file
    midi_data = pretty_midi.PrettyMIDI(midi_file)

    # Time interval for calculating features
    interval = 1.0  # 1 second
    times = np.arange(0, midi_data.get_end_time(), interval)

    # Create arrays for storing time series data
    pitch = np.zeros(len(times))
    volume = np.zeros(len(times))
    note_density = np.zeros(len(times))
    tempo = np.zeros(len(times))

    # Calculate time series data for each feature
    for i, t in enumerate(times):
        # Get notes that are playing at this time
        notes = [note for note in midi_data.instruments[0].notes if note.start <= t < note.end]

        # Calculate average pitch
        if notes:
            pitch[i] = np.mean([note.pitch for note in notes])

        # Calculate note density (notes per second)
        note_density[i] = len(notes) / interval

        # Calculate average volume
        if notes:
            volume[i] = np.mean([note.velocity for note in notes])

    # Calculate rhythmic complexity (variance in inter-onset intervals)
    inter_onset_intervals = np.diff([note.start for note in midi_data.instruments[0].notes])
    rhythmic_complexity = np.var(inter_onset_intervals)

    # Calculate tempo for each moment in time
    tempo_changes = midi_data.get_tempo_changes()
    tempo = np.interp(times, tempo_changes[0], tempo_changes[1])

    return times, pitch, note_density, volume, rhythmic_complexity, tempo

def process_composer_data():
    # Initialize DataFrame
    df = pd.DataFrame(columns=["Composer","Times", "Pitch", "Note_Density", "Volume", 
                               "Rhythmic_Complexity", "Tempo"])

    # Iterate over all composer directories
    for composer_dir in glob.glob(os.path.join(base_dir, '*')):
        # Get the composer's name
        composer_name = os.path.basename(composer_dir)
        print(f"Processing {composer_name} MIDI files...")

        # Iterate over all MIDI files in composer's directory
        for midi_file in glob.glob(os.path.join(composer_dir, '*.mid')):
            print(f"Processing {midi_file}...")
            try:
                times, pitch, note_density, volume, rhythmic_complexity, tempo = calculate_features(midi_file)
                # Append to DataFrame
                df = df.append({"Composer": composer_name, "Times": times, "Pitch": pitch, 
                                "Note_Density": note_density, "Volume": volume, 
                                "Rhythmic_Complexity": rhythmic_complexity, 
                                "Tempo": tempo}, 
                               ignore_index=True)
            except Exception as e:
                print(f"Error processing {midi_file}: {str(e)}")
    
    # Write the DataFrame to a pickle file
    df.to_pickle(base_dir + "/" + pickle_file_name)
    
    return df

In [19]:
# Data Processing
# Feature extraction

pickle_file = base_dir + "/" + pickle_file_name
# Check if the pickle file exists
if not os.path.exists(pickle_file):
    print("Music Data not Pickled, creating dataset using feature extract.")
    df = process_composer_data()
else:
    # Open the pickle file in binary mode and load the data
    with open(pickle_file, 'rb') as file:
        data = pickle.load(file)

    # Create a DataFrame from the loaded data
    df = pd.DataFrame(data)

    # Now you have your DataFrame ready for use
    print(df.head())

Music Data not Pickled, creating dataset using feature extract.
Processing bach MIDI files...
Processing train\bach\bach342.mid...
Processing train\bach\bach343.mid...


C:\Users\chris\AppData\Local\Temp\ipykernel_11408\1506090990.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Composer": composer_name, "Times": times, "Pitch": pitch,
C:\Users\chris\AppData\Local\Temp\ipykernel_11408\1506090990.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Composer": composer_name, "Times": times, "Pitch": pitch,
C:\Users\chris\AppData\Local\Temp\ipykernel_11408\1506090990.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Composer": composer_name, "Times": times, "Pitch": pitch,
C:\Users\chris\AppData\Local\Temp\ipykernel_11408\1506090990.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

Processing train\bach\bach345.mid...
Processing train\bach\bach346.mid...
Processing train\bach\bach347.mid...
Processing train\bach\bach350.mid...


C:\Users\chris\AppData\Local\Temp\ipykernel_11408\1506090990.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Composer": composer_name, "Times": times, "Pitch": pitch,
C:\Users\chris\AppData\Local\Temp\ipykernel_11408\1506090990.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Composer": composer_name, "Times": times, "Pitch": pitch,
C:\Users\chris\AppData\Local\Temp\ipykernel_11408\1506090990.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Composer": composer_name, "Times": times, "Pitch": pitch,
C:\Users\chris\AppData\Local\Temp\ipykernel_11408\1506090990.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

Processing train\bach\bach351.mid...
Processing train\bach\bach352.mid...
Processing train\bach\bach353.mid...
Processing train\bach\bach355.mid...
Processing train\bach\bach357.mid...


...

C:\Users\chris\AppData\Local\Temp\ipykernel_11408\1506090990.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Composer": composer_name, "Times": times, "Pitch": pitch,
C:\Users\chris\AppData\Local\Temp\ipykernel_11408\1506090990.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Composer": composer_name, "Times": times, "Pitch": pitch,
C:\Users\chris\AppData\Local\Temp\ipykernel_11408\1506090990.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Composer": composer_name, "Times": times, "Pitch": pitch,


Processing train\schumann\schumann237.mid...
Processing train\schumann\schumann239.mid...
Processing train\schumann\schumann241.mid...


C:\Users\chris\AppData\Roaming\Python\Python39\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Processing train\schumann\schumann242.mid...


C:\Users\chris\AppData\Local\Temp\ipykernel_11408\1506090990.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Composer": composer_name, "Times": times, "Pitch": pitch,
C:\Users\chris\AppData\Local\Temp\ipykernel_11408\1506090990.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Composer": composer_name, "Times": times, "Pitch": pitch,


In [20]:
# Data Pre-processing
print(df.head())
print(df.info())

  Composer                                              Times  \
0     bach  [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...   
1     bach  [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...   
2     bach  [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...   
3     bach  [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...   
4     bach  [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...   

                                               Pitch  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 73.0,...   
1  [0.0, 78.0, 78.0, 0.0, 66.0, 66.0, 0.0, 0.0, 6...   
2  [0.0, 67.0, 43.0, 0.0, 67.0, 0.0, 43.0, 55.0, ...   
3  [0.0, 0.0, 50.0, 74.0, 0.0, 74.0, 0.0, 74.0, 0...   
4  [0.0, 63.0, 69.0, 0.0, 0.0, 69.0, 0.0, 45.0, 0...   

                                        Note_Density  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...   
1  [0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, ...   
2  [0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, ...   
3  [0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0,

In [ ]:
# Feature Extraction

In [ ]:
# Model Building

In [ ]:
# Model Training

In [ ]:
# Model Evaluation

In [ ]:
# Model Optimization